Extracție Commits

Vom colecta pentru fiecare commit: sha, autor, date, LoC, etc.

In [ ]:
%run ./utils.ipynb

In [ ]:
from github.GithubException import GithubException
import pandas as pd

In [ ]:
repos = read_repo_list()   # acum shallow_data.csv e în același folder
print(f"Am {len(repos)} repo-uri. Primele 5:")
print(repos[:5])

In [ ]:
# Test read_repo_list()
repos = read_repo_list(csv_path="shallow_data.csv")
print(f"Am {len(repos)} repo-uri. Primele 5:", repos[:5])

In [ ]:
# Test ensure_repo_folder()
test_repo = repos[0]
folder = ensure_repo_folder(test_repo)
print("Folder creat pentru", test_repo, "→", folder)

Metoda pentru extragere de commits

In [ ]:
def extract_commits(repo_full_name, dest_folder, max_commits=None):
    repo = gh.get_repo(repo_full_name)
    rows = []
    for i, commit in enumerate(repo.get_commits()):
        if max_commits and i >= max_commits:
            break
        stats     = commit.stats
        author    = commit.author
        committer = commit.committer
        rows.append({
            "repo_full_name":    repo_full_name,
            "sha":               commit.sha,
            "html_url":          commit.html_url,
            "author_login":      author.login    if author    else None,
            "author_id":         author.id       if author    else None,
            "author_type":       author.type     if author    else None,
            "authored_date":     commit.commit.author.date.isoformat(),
            "committer_login":   committer.login if committer else None,
            "committed_date":    commit.commit.committer.date.isoformat(),
            "message":           commit.commit.message,
            "additions":         stats.additions,
            "deletions":         stats.deletions,
            "total_changes":     stats.total,
            "files_changed_count": len(list(commit.files)),
            "parent_shas":       ",".join(p.sha for p in commit.parents)
        })
    df = pd.DataFrame(rows)
    df.to_csv(dest_folder/"commits.csv", index=False)
    return df

In [ ]:
# Test extract_commits pe un singur repo
df_test = extract_commits(test_repo, folder, max_commits=10)
print("Test shape:", df_test.shape)
df_test.head()

In [ ]:
for full_name in repos:
    log(f"Commits → {full_name}")
    folder = ensure_repo_folder(full_name)
    extract_commits(full_name, folder)